Import Libraries

In [1]:
import numpy as np
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
from tensorflow.keras.applications import ResNet50, InceptionV3, DenseNet201
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from datetime import datetime
import time

Preparing Data

In [2]:
# Define directory paths for infected and uninfected images
base_dir = 'D:/R001/cell_images'  # Update this with the actual base directory path

# Set up ImageDataGenerator for loading images in batches
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)  # Normalize images and set aside 20% for validation

# Load training data in batches
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,  # Adjust batch size as needed based on memory
    class_mode='binary',
    subset='training'
)

# Load validation data in batches
validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.


Load Model ResNet

In [3]:
loaded_model_resnet = load_model('model_resnet5ep.keras')

In [5]:
loss, accuracy = loaded_model_resnet.evaluate(validation_generator)

# Print the accuracy
print(f'Accuracy of the loaded model: {accuracy * 100:.2f}%')

C:\Users\Hamza\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


173/173 ━━━━━━━━━━━━━━━━━━━━ 387s 2s/step - accuracy: 0.9533 - loss: 0.1609
Accuracy of the loaded model: 95.68%


Load Model Inception

In [4]:
loaded_model_inception = load_model('model_inception5ep.keras')

In [7]:
loss, accuracy = loaded_model_inception.evaluate(validation_generator)

# Print the accuracy
print(f'Accuracy of the loaded model: {accuracy * 100:.2f}%')

173/173 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.9504 - loss: 0.1911
Accuracy of the loaded model: 94.52%


DenseNet

In [5]:
loaded_model_densenet = load_model('model_densenet5ep.keras')

In [9]:
loss, accuracy = loaded_model_densenet .evaluate(validation_generator)

# Print the accuracy
print(f'Accuracy of the loaded model: {accuracy * 100:.2f}%')

173/173 ━━━━━━━━━━━━━━━━━━━━ 471s 3s/step - accuracy: 0.9598 - loss: 0.1355
Accuracy of the loaded model: 95.74%


Using Stacking with rf

In [6]:
def predict_with_generator(model, generator):
    preds = []
    labels = []
    for batch, label in generator:
        pred = model.predict(batch)
        preds.extend(pred)
        labels.extend(label)
        if len(preds) >= generator.samples:
            break
    return np.array(preds)[:generator.samples], np.array(labels)[:generator.samples]

# Time tracking: Start
start_time = time.time()

resnet_preds, y_test = predict_with_generator(loaded_model_resnet, validation_generator)
inception_preds, _ = predict_with_generator(loaded_model_inception, validation_generator)
densenet_preds, _ = predict_with_generator(loaded_model_densenet, validation_generator)

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 

In [7]:
# Assuming you have already loaded the models and generated predictions (resnet_preds, inception_preds, densenet_preds)
# Stack the predictions together
stacked_predictions = np.column_stack((resnet_preds, inception_preds, densenet_preds))

# Train a RF model as the meta-model
meta_model = RandomForestClassifier()
meta_model.fit(stacked_predictions, y_test)

# Make predictions with the meta-model
stacked_final_preds = meta_model.predict(stacked_predictions)

In [8]:
# Evaluate the performance of the stacked model
accuracy = accuracy_score(y_test, stacked_final_preds)
precision = precision_score(y_test, stacked_final_preds)
recall = recall_score(y_test, stacked_final_preds)
f1 = f1_score(y_test, stacked_final_preds)

In [9]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.997459165154265
Precision: 0.9967379485320769
Recall: 0.9981851179673321
F1 Score: 0.9974610083424011


Prediction

In [10]:
# Function to preprocess a single input image
def preprocess_single_image(image_path, target_size=(64, 64)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to predict a single image and log the timestamp
def predict_single_image(image_path, models, meta_model):
    resnet_model, inception_model, densenet_model = models
    
    # Preprocess the input image
    preprocessed_image = preprocess_single_image(image_path)

    # Get individual model predictions
    resnet_pred = resnet_model.predict(preprocessed_image)
    inception_pred = inception_model.predict(preprocessed_image)
    densenet_pred = densenet_model.predict(preprocessed_image)

    # Stack predictions
    stacked_pred = np.column_stack((resnet_pred, inception_pred, densenet_pred))

    # Predict using meta-model
    start_time = time.time()  # Track start time
    final_prediction = meta_model.predict(stacked_pred)
    end_time = time.time()  # Track end time
    
    # Decode the result
    class_label = "Infected" if final_prediction[0] == 1 else "Uninfected"
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    time_taken = end_time - start_time

    # Print and return the result
    print(f"[{timestamp}] Prediction: {class_label} (Time taken: {time_taken:.4f} seconds)")
    return class_label, time_taken



# List of preloaded models
models = (loaded_model_resnet, loaded_model_inception, loaded_model_densenet)

Predicted Images

In [11]:
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from datetime import datetime
import time

# Function to preprocess a single input image
def preprocess_single_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)  # Resize the image
    img_array = img_to_array(img)  # Convert to numpy array
    img_array = img_array.astype('float32') / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to predict the class of a single image
def predict_single_image(image_path, models, meta_model):
    # Extract models
    resnet_model, inception_model, densenet_model = models

    # Preprocess the input image
    preprocessed_image = preprocess_single_image(image_path)

    # Get predictions from individual models
    resnet_pred = resnet_model.predict(preprocessed_image).flatten()
    inception_pred = inception_model.predict(preprocessed_image).flatten()
    densenet_pred = densenet_model.predict(preprocessed_image).flatten()

    # Stack the predictions into a single input for the meta-model
    stacked_pred = np.column_stack((resnet_pred, inception_pred, densenet_pred))

    # Time tracking
    start_time = time.time()
    final_prediction = meta_model.predict(stacked_pred)  # Predict with meta-model
    end_time = time.time()

    # Decode the prediction
    class_label = "Un-Infected" if final_prediction[0] == 1 else "Infected"
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    time_taken = end_time - start_time

    # Print and return the result
    print(f"[{timestamp}] Prediction: {class_label} (Time taken: {time_taken:.4f} seconds)")
    return class_label, time_taken



# List of loaded models
models = (loaded_model_resnet, loaded_model_inception, loaded_model_densenet)

In [17]:
# Example usage
# Path to the input image
input_image_path = "1.jpg"

# Predict the class of the input image
predict_single_image(input_image_path, models, meta_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step
[2025-01-12 23:45:25] Prediction: Infected (Time taken: 0.0104 seconds)


('Infected', 0.010408163070678711)

In [18]:
input_image_path = "2.jpg"
predict_single_image(input_image_path, models, meta_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 559ms/step
[2025-01-12 23:45:37] Prediction: Infected (Time taken: 0.0099 seconds)


('Infected', 0.009852170944213867)

In [19]:
input_image_path = "3.jpg"
predict_single_image(input_image_path, models, meta_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step
[2025-01-12 23:45:38] Prediction: Un-Infected (Time taken: 0.0106 seconds)


('Un-Infected', 0.010577917098999023)

In [20]:
input_image_path = "4.jpg"
predict_single_image(input_image_path, models, meta_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step
[2025-01-12 23:45:39] Prediction: Un-Infected (Time taken: 0.0072 seconds)


('Un-Infected', 0.0072329044342041016)

In [21]:
input_image_path = "5.png"
predict_single_image(input_image_path, models, meta_model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step
[2025-01-12 23:45:41] Prediction: Infected (Time taken: 0.0141 seconds)


('Infected', 0.014103889465332031)